# 09wk-2: 중간고사

최규빈  
2024-05-01

# 6. 나는 SOLO – 5의 다른구현 (torch)

## A. 임베딩레이어 이용

In [172]:
#df_view = pd.read_csv('https://raw.githubusercontent.com/guebin/DL2024/main/posts/solo.csv',index_col=0)
df = df_view.stack().reset_index().set_axis(['W','M','y'],axis=1)
w = {'옥순':0, '영자':1, '정숙':2, '영숙':3, '순자':4, '현숙':5}
m = {'영식':0, '영철':1, '영호':2, '광수':3, '상철':4, '영수':5}
X1 = torch.tensor(df['W'].map(w)) # length-n vector 
X2 = torch.tensor(df['M'].map(m)) # length-n vector 
# E1 = torch.nn.functional.one_hot(X1).float()
# E2 = torch.nn.functional.one_hot(X2).float()
y = torch.tensor(df['y']).float().reshape(-1,1)

In [178]:
#--#
torch.manual_seed(43052)
e1 = torch.nn.Embedding(num_embeddings=6,embedding_dim=2)
b1 = torch.nn.Embedding(num_embeddings=6,embedding_dim=1)
e2 = torch.nn.Embedding(num_embeddings=6,embedding_dim=2)
b2 = torch.nn.Embedding(num_embeddings=6,embedding_dim=1)
sig = torch.nn.Sigmoid()
loss_fn = torch.nn.MSELoss()
optimizr = torch.optim.Adam(
    list(e1.parameters())+list(e2.parameters())+list(b1.parameters())+list(b2.parameters()),
    lr = 0.01
)
#--#
for epoc in range(300):
    ## step1 
    W_features = e1(X1) 
    W_bias = b1(X1)
    M_features = e2(X2) 
    M_bias = b2(X2)
    dot = (W_features * M_features + W_bias + M_bias).sum(axis=1)
    yhat = (sig(dot)*5).reshape(-1,1)
    ## step2 
    loss = loss_fn(yhat,y)
    ## step3 
    loss.backward()
    ## step4 
    optimizr.step()
    optimizr.zero_grad()

In [179]:
df_ebdspace = pd.DataFrame(torch.concat([W_features.data,W_bias.data, M_features.data, M_bias.data],axis=1))
df_ebdspace.columns = ['여성잠재특징1','여성잠재특징2','여성바이어스','남성잠재특징1','남성잠재특징2','남성바이어스']
df_yhat = pd.DataFrame(yhat.data,columns=['yhat'])
pd.concat([df,df_yhat,df_ebdspace],axis=1)

## B. net이용

# 7. 나는 SOLO – 5의 다른구현 (fastai)

## A. 코딩..

In [800]:
dls = fastai.collab.CollabDataLoaders.from_df(df,bs=2,valid_pct = 2/24)
lrnr = fastai.collab.collab_learner(dls, n_factors=2, y_range=(0,5))
lrnr.fit(30,lr=0.1)

`-` 예측

In [806]:
df_test = pd.DataFrame({'W':['정숙']*6, 'M':['영식','영철','영호','광수','상철','영수']})
df_test

In [819]:
yhat,_ = lrnr.get_preds(dl = dls.test_dl(df_test))
df_test.assign(yhat = yhat)

-   적절해보임

## B. `fastai` 의 net, loss 등을 구경

In [823]:
lrnr.model

In [824]:
lrnr.forward?? 

-   우리가 만든 모델이랑 같음!

In [826]:
lrnr.loss_func 